In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# ls drive/MyDrive/'Final_Project_ICH'/'CSV_file'/

In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt

# Очистка данных в таблице **Calls**
- преобразование формата данных Id, CONTACTID строчный форомат(object)
- преобразование дат в формат datetime
- проверка дубликатов и их удаление
- удаление пустых(не релевантных) столбцов
- просмотр уникальных значений
- проверка значений NaN
- замена типа данных на категориальные Call Owner Name, Call Type, Call Status, Outgoing Call Status, Scheduled in CRM
- заполнение/замена значений категориальных NaN = Unknown
- контроль данных и значений

In [4]:
calls_df = pd.read_csv('drive/MyDrive/Final_Project_ICH/CSV_file/Calls.csv', dtype={"Id": 'str', 'CONTACTID': 'str'})
#calls_df.info()
calls_df.head()

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Dialled Number,Outgoing Call Status,Scheduled in CRM,Tag
0,5805028000000805001,30.06.2023 08:43,John Doe,NaN,Inbound,171.0,Received,NaN,NaN,NaN,NaN
1,5805028000000768006,30.06.2023 08:46,John Doe,NaN,Outbound,28.0,Attended Dialled,NaN,Completed,False,NaN
2,5805028000000764027,30.06.2023 08:59,John Doe,NaN,Outbound,24.0,Attended Dialled,NaN,Completed,False,NaN
3,5805028000000787003,30.06.2023 09:20,John Doe,5805028000000645014,Outbound,6.0,Attended Dialled,NaN,Completed,False,NaN
4,5805028000000768019,30.06.2023 09:30,John Doe,5805028000000645014,Outbound,11.0,Attended Dialled,NaN,Completed,False,NaN


In [5]:
# Преобразование формата дат
calls_df["Call Start Time"] = pd.to_datetime(calls_df["Call Start Time"], format="%d.%m.%Y %H:%M", errors='coerce')

In [6]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          95874 non-null  object        
 1   Call Start Time             95874 non-null  datetime64[ns]
 2   Call Owner Name             95874 non-null  object        
 3   CONTACTID                   91941 non-null  object        
 4   Call Type                   95874 non-null  object        
 5   Call Duration (in seconds)  95791 non-null  float64       
 6   Call Status                 95874 non-null  object        
 7   Dialled Number              0 non-null      float64       
 8   Outgoing Call Status        86875 non-null  object        
 9   Scheduled in CRM            86875 non-null  object        
 10  Tag                         0 non-null      float64       
dtypes: datetime64[ns](1), float64(3), object(7)
memory usa

In [7]:
# удаление пустых колонок
calls_df.drop(['Tag', 'Dialled Number'], axis=1, inplace=True)

In [8]:
# проверка дубликатов
calls_df[calls_df.duplicated()]

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM


In [9]:
calls_df['Scheduled in CRM'].value_counts()

,count
Scheduled in CRM,
False,86733
True,142


In [10]:
calls_df['Call Type'].value_counts()

,count
Call Type,
Outbound,86875
Missed,5921
Inbound,3078


In [11]:
calls_df['Call Status'].value_counts()

,count
Call Status,
Attended Dialled,70703
Unattended Dialled,16030
Missed,5922
Received,3077
Overdue,60
Scheduled Attended Delay,22
Cancelled,20
Scheduled Unattended Delay,17
Scheduled Attended,14


In [12]:
calls_df['Outgoing Call Status'].value_counts()

,count
Outgoing Call Status,
Completed,86792
Overdue,60
Cancelled,20
Scheduled,3


In [13]:
calls_df.isnull().sum()

,0
Id,0
Call Start Time,0
Call Owner Name,0
CONTACTID,3933
Call Type,0
Call Duration (in seconds),83
Call Status,0
Outgoing Call Status,8999
Scheduled in CRM,8999


In [14]:
calls_df.shape

(95874, 9)

In [15]:
calls_df[calls_df['CONTACTID'].isnull()]

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM
0,5805028000000805001,2023-06-30 08:43:00,John Doe,NaN,Inbound,171.0,Received,NaN,NaN
1,5805028000000768006,2023-06-30 08:46:00,John Doe,NaN,Outbound,28.0,Attended Dialled,Completed,False
2,5805028000000764027,2023-06-30 08:59:00,John Doe,NaN,Outbound,24.0,Attended Dialled,Completed,False
7,5805028000000879006,2023-07-03 13:06:00,Jane Smith,NaN,Outbound,0.0,Unattended Dialled,Completed,False
8,5805028000000870005,2023-07-03 13:08:00,Jane Smith,NaN,Outbound,40.0,Attended Dialled,Completed,False
...,...,...,...,...,...,...,...,...,...
95852,5805028000056849471,2024-06-21 15:06:00,Ulysses Adams,NaN,Outbound,7.0,Attended Dialled,Completed,False
95853,5805028000056859477,2024-06-21 15:07:00,Ulysses Adams,NaN,Inbound,407.0,Received,NaN,NaN
95861,5805028000056876318,2024-06-21 15:20:00,Eva Kent,NaN,Outbound,5.0,Attended Dialled,Completed,False
95868,5805028000056912329,2024-06-21 15:30:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True


In [16]:
# Получение уникальных значений в столбце 'Contact Owner Name'
unique_values = calls_df['Call Owner Name'].unique()
unique_values

array(['John Doe', 'Jane Smith', 'Alice Johnson', 'Bob Brown',
       'Charlie Davis', 'Diana Evans', 'Ethan Harris', 'Fiona Jackson',
       'George King', 'Hannah Lee', 'Ian Miller', 'Julia Nelson',
       'Kevin Parker', 'Laura Quinn', 'Mason Roberts', 'Nina Scott',
       'Oliver Taylor', 'Paula Underwood', 'Quincy Vincent',
       'Rachel White', 'Sam Young', 'Tina Zhang', 'Ulysses Adams',
       'Victor Barnes', 'Wendy Clark', 'Xander Dean', 'Yara Edwards',
       'Zachary Foster', 'Amy Green', 'Ben Hall', 'Cara Iverson',
       'Derek James', 'Eva Kent'], dtype=object)

In [17]:
calls_df['Call Owner Name'].nunique()

33

In [18]:
calls_df['CONTACTID'].nunique()

15214

In [19]:
calls_df[calls_df['Call Duration (in seconds)'].isnull()]

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM
21711,5805028000012419809,2023-11-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Cancelled,Cancelled,True
36095,5805028000012419971,2024-01-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Overdue,Overdue,True
36096,5805028000012419512,2024-01-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Overdue,Overdue,True
37600,5805028000026957356,2024-01-19 10:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
37607,5805028000026466529,2024-01-19 10:30:00,Victor Barnes,5805028000021065227,Outbound,NaN,Overdue,Overdue,True
...,...,...,...,...,...,...,...,...,...
89073,5805028000053636072,2024-06-05 17:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
89257,5805028000053651109,2024-06-06 07:00:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
92959,5805028000055092068,2024-06-14 16:00:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True
93676,5805028000055626456,2024-06-17 10:00:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True


In [20]:
calls_df[calls_df['Call Status']== 'Scheduled']

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM
92959,5805028000055092068,2024-06-14 16:00:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True
93676,5805028000055626456,2024-06-17 10:00:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True
95868,5805028000056912329,2024-06-21 15:30:00,Victor Barnes,NaN,Outbound,NaN,Scheduled,Scheduled,True


In [21]:
# Заменяем значения в столбце "Call Duration (in seconds)" на 0, если "Call Status" == "Overdue"
calls_df.loc[calls_df['Call Status'] == 'Scheduled', 'Call Duration (in seconds)'] = 0

In [22]:
calls_df[calls_df['Call Status']== 'Cancelled']

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM
21711,5805028000012419809,2023-11-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Cancelled,Cancelled,True
39785,5805028000028203333,2024-01-25 12:00:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
42685,5805028000012419105,2024-02-01 15:00:00,Kevin Parker,NaN,Outbound,NaN,Cancelled,Cancelled,True
45662,5805028000012635140,2024-02-10 18:00:00,Kevin Parker,NaN,Outbound,NaN,Cancelled,Cancelled,True
50216,5805028000012419919,2024-02-25 15:00:00,Kevin Parker,NaN,Outbound,NaN,Cancelled,Cancelled,True
67870,5805028000043554064,2024-04-12 16:30:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
67871,5805028000043267197,2024-04-12 16:30:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
68597,5805028000043831431,2024-04-15 17:30:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
71214,5805028000045147916,2024-04-21 15:45:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True
71382,5805028000045343826,2024-04-22 09:30:00,Victor Barnes,NaN,Outbound,NaN,Cancelled,Cancelled,True


In [23]:
# Заменяем значения в столбце "Call Duration (in seconds)" на 0, если "Call Status" == "Overdue"
calls_df.loc[calls_df['Call Status'] == 'Cancelled', 'Call Duration (in seconds)'] = 0

In [24]:
calls_df[calls_df['Call Status']== 'Overdue']

,Id,Call Start Time,Call Owner Name,CONTACTID,Call Type,Call Duration (in seconds),Call Status,Outgoing Call Status,Scheduled in CRM
36095,5805028000012419971,2024-01-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Overdue,Overdue,True
36096,5805028000012419512,2024-01-15 15:00:00,Kevin Parker,NaN,Outbound,NaN,Overdue,Overdue,True
37600,5805028000026957356,2024-01-19 10:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
37607,5805028000026466529,2024-01-19 10:30:00,Victor Barnes,5805028000021065227,Outbound,NaN,Overdue,Overdue,True
37623,5805028000026957294,2024-01-19 11:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
37972,5805028000027189639,2024-01-19 19:40:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
38236,5805028000026738099,2024-01-21 13:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
38237,5805028000027189317,2024-01-21 13:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True
38383,5805028000027542172,2024-01-22 10:00:00,Victor Barnes,5805028000026965161,Outbound,NaN,Overdue,Overdue,True
38439,5805028000027189233,2024-01-22 13:00:00,Victor Barnes,NaN,Outbound,NaN,Overdue,Overdue,True


In [25]:
# Заменяем значения в столбце "Call Duration (in seconds)" на 0, если "Call Status" == "Overdue"
calls_df.loc[calls_df['Call Status'] == 'Overdue', 'Call Duration (in seconds)'] = 0

In [26]:
calls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          95874 non-null  object        
 1   Call Start Time             95874 non-null  datetime64[ns]
 2   Call Owner Name             95874 non-null  object        
 3   CONTACTID                   91941 non-null  object        
 4   Call Type                   95874 non-null  object        
 5   Call Duration (in seconds)  95874 non-null  float64       
 6   Call Status                 95874 non-null  object        
 7   Outgoing Call Status        86875 non-null  object        
 8   Scheduled in CRM            86875 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 6.6+ MB


In [27]:
category_columns = ['Call Owner Name', 'Call Type', 'Call Status', 'Outgoing Call Status', 'Scheduled in CRM']
for col in category_columns:
    calls_df[col] = calls_df[col].fillna('Unknown')

In [28]:
calls_df['Scheduled in CRM'] = calls_df['Scheduled in CRM'].astype(str)

In [29]:
calls_df.info()
calls_df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95874 entries, 0 to 95873
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Id                          95874 non-null  object        
 1   Call Start Time             95874 non-null  datetime64[ns]
 2   Call Owner Name             95874 non-null  object        
 3   CONTACTID                   91941 non-null  object        
 4   Call Type                   95874 non-null  object        
 5   Call Duration (in seconds)  95874 non-null  float64       
 6   Call Status                 95874 non-null  object        
 7   Outgoing Call Status        95874 non-null  object        
 8   Scheduled in CRM            95874 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 6.6+ MB


,0
Id,0
Call Start Time,0
Call Owner Name,0
CONTACTID,3933
Call Type,0
Call Duration (in seconds),0
Call Status,0
Outgoing Call Status,0
Scheduled in CRM,0


# **Описательная статистика**

In [30]:
# сводная статистика для числовых столбцов
numeric_stats = calls_df.describe()
display(numeric_stats)

,Call Start Time,Call Duration (in seconds)
count,95874,95874.000000
mean,2024-02-05 12:34:44.369484800,164.834439
min,2023-06-30 08:43:00,0.000000
25%,2023-11-24 11:07:30,4.000000
50%,2024-02-19 12:19:30,8.000000
75%,2024-04-22 19:10:00,97.000000
max,2024-06-21 15:31:00,7625.000000
std,NaN,401.266369


In [31]:
calls_df['CONTACTID'].value_counts()

,count
CONTACTID,
5805028000003329100,94
5805028000008246089,72
5805028000007866898,70
5805028000001880249,67
5805028000002799040,57
...,...
5805028000021705633,1
5805028000021694761,1
5805028000056892018,1


In [32]:
# Вывод количества уникальных значений и их частоты для каждой категориальной переменной

for col in calls_df.select_dtypes(include=['object']).columns:
    print(f"\nСтатистика по {col}:")
    print(calls_df[col].value_counts().head(100))  # Выводим 100 значений
    print("-" * 50)  # Разделитель для удобства чтения


Статистика по Id:
Id
5.80503E+18            3
5805028000041757323    1
5805028000041753189    1
5805028000041733204    1
5805028000041734423    1
                      ..
5805028000041755675    1
5805028000041752614    1
5805028000041766422    1
5805028000041753328    1
5805028000041753316    1
Name: count, Length: 100, dtype: int64
--------------------------------------------------

Статистика по Call Owner Name:
Call Owner Name
Yara Edwards       9059
Julia Nelson       7446
Ian Miller         7215
Charlie Davis      7213
Diana Evans        6857
Ulysses Adams      6085
Amy Green          5982
Nina Scott         5581
Victor Barnes      5439
Kevin Parker       5406
Paula Underwood    4580
Quincy Vincent     4384
Jane Smith         3753
Cara Iverson       3300
John Doe           2986
Ben Hall           2947
Alice Johnson      1251
Mason Roberts      1166
Derek James         948
George King         850
Zachary Foster      523
Eva Kent            498
Fiona Jackson       470
Sam Young    

In [33]:
calls_df.to_csv('drive/MyDrive/Final_Project_ICH/Clean_file/CallsClean.csv')
calls_df.to_parquet('drive/MyDrive/Final_Project_ICH/Clean_file/CallsClean.parquet')